In [1]:
def make_Pol_file(phot,calib,eff=True,inst=True,off=True,theta_off=False, etheta_off = False):
    df = pd.read_csv(phot)
    order = np.arange(0,len(df),4)    
    FILE = df['filename'].values

    Pol_Log = pd.DataFrame({})
    for z in order:
    
        SET = [FILE[z],FILE[z+1], FILE[z+2], FILE[z+3]]
        kappa, S = [],[]

        JD_av = np.mean(df['JD'].values[z:z+4])
        psANG_av = np.mean(df['PsANG [deg]'].values[z:z+4])
        file_name = SET[0].split('/')[-1].split('.')[0] +' ~ '+SET[3].split('/')[-1].split('_')[-1]
        OBJECT = df['Object'].values[0]
        UT_av = Time(JD_av, format='jd').isot
        Date_obs = UT_av.split('T')[0]
        UT_obs = UT_av.split('T')[-1][:5]
        aperture_radius = np.mean(df['Aper_radius [pix]'].values[z:z+4])
        ap = np.mean(df['Aper_scale'].values[z:z+4])
        FWHM_o =  np.mean(df['FWHM_ordi [pix]'].values[z:z+4])
        FWHM_e = np.mean(df['FWHM_extra [pix]'].values[z:z+4])
        AIRMASS = np.mean(df['AIRMASS'].values[z:z+4])
        EXP = np.mean(df['EXPTIME'].values[z:z+4])
        pA = np.mean(df['alpha [deg]'].values[z:z+4])
        psANG = np.mean(df['PsANG [deg]'].values[z:z+4])
        snr = np.mean(df['SNR'].values[z:z+4])
        delta = np.mean(df['delta'].values[z:z+4])
        r = np.mean(df['r'].values[z:z+4])
        INR_STR = df['INR-STR'].values[z:z+4]
        INR_END = df['INR-END'].values[z:z+4]
        INSTPA = np.mean(df['INST-PA'].values[z:z+4])
        INSROT = np.mean(df['INSROT'].values[z:z+4])
        INROFF = np.mean(df['INROFF'].values[z:z+4])
        V = np.nanmean(df['V'].values[z:z+4])
        

        cen_o = df['Cen_o'].values[z]
        cen_e = df['Cen_e'].values[z]
        y_diff = float(cen_e.split(',')[0][1:]) - float(cen_o.split(',')[0][1:])
        x_diff = float(cen_e.split(',')[1][:-1]) - float(cen_o.split(',')[1][:-1])
        
        ### WHERE THE TILTED ANGLE IS DERVIED## 
        tilted_ang = np.round(np.rad2deg(np.arctan2(x_diff,y_diff)),2)
        if tilted_ang <0:
            tilted_ang = tilted_ang+360

        Type = df['Type'].values[0]
        flag_ls = ''
        FILTER = df['Filter'].values[0]
        eff_flag = None
        inst_flag = None
        off_flag = None


        for i, ang in enumerate([0,45,22.5,67.5]):
            file = SET[i]
            df_ = df[df['filename']==file]
            HWP = df_['HWPANG'].values[0]
            FILTER = df_['Filter'].values[0]
            if HWP != ang:
                print('HWPANG order is 0,45, 22.5, 67.5?')
            Flux_extra = df_['Flux_extra [e]'].values[0]
            Flux_ordi = df_['Flux_ordi [e]'].values[0]
            ERR_extra = df_['eFlux_extra [e]'].values[0]
            ERR_ordi = df_['eFlux_ordi [e]'].values[0]
            kappa.append(Flux_extra/Flux_ordi)
            S_ret = (ERR_extra/(Flux_extra))**2 + (ERR_ordi/(Flux_ordi))**2
            S.append(S_ret)
        
        Msi = MSI(psANG,FILTER,calib)
        q,u,q_ran,q_sys,u_ran,u_sys = Msi.Get_qu(kappa,S)
        q_raw,u_raw,q_ran_raw,q_sys_raw,u_ran_raw,u_sys_raw = q,u,q_ran,q_sys,u_ran,u_sys
        q_eff,u_eff,q_ran_eff,q_sys_eff,u_ran_eff,u_sys_eff = np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
        q_inst,u_inst,q_ran_inst,q_sys_inst,u_ran_inst,u_sys_inst = np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
        q_off,u_off,q_ran_off,q_sys_off,u_ran_off,u_sys_off = np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
        
        if eff==True:
            q,u,q_ran,q_sys,u_ran,u_sys,eff_, err_eff_ = Msi.Correct_Efficiency(q,u,
                                                                                q_ran,q_sys,
                                                                                u_ran,u_sys, 
                                                                                verbose=True)
            q_eff,u_eff,q_ran_eff,q_sys_eff,u_ran_eff,u_sys_eff = q,u,q_ran,q_sys,u_ran,u_sys
            eff_flag = (eff_, err_eff_)
        if inst==True:
            q,u,q_ran,q_sys,u_ran,u_sys,q_inst_c, err_qin_c, u_inst_c, err_uin_c = Msi.Correct_Inst_Pol(q,u,
                                                                                                        q_ran,q_sys,
                                                                                                        u_ran,u_sys,
                                                                                                        INR_STR,
                                                                                                        INR_END,
                                                                                                        tilted_ang,
                                                                                                        verbose=True)
            q_inst,u_inst,q_ran_inst,q_sys_inst,u_ran_inst,u_sys_inst = q,u,q_ran,q_sys,u_ran,u_sys
            inst_flag = (q_inst_c, err_qin_c, u_inst_c, err_uin_c)
        if off==True:
            q,u,q_ran,q_sys,u_ran,u_sys,off_,err_off = Msi.Transform_CelestialCoord(q,u,
                                                                                    q_ran,q_sys,
                                                                                    u_ran,u_sys,
                                                                                    -0.52,
                                                                                    tilted_ang,
                                                                                    verbose=True)
            q_off,u_off,q_ran_off,q_sys_off,u_ran_off,u_sys_off=q,u,q_ran,q_sys,u_ran,u_sys
            off_flag = (off_, err_off)
            
        eq, eu = (q_ran**2 + q_sys**2)**0.5,(u_ran**2 + u_sys**2)**0.5
        P = np.sqrt(q**2 + u**2)
        P_ran = np.sqrt( (q*q_ran)**2 + (u*u_ran)**2 )/P
        P_sys = np.sqrt( (q*q_sys)**2 + (u*u_sys)**2 )/P
        P_err = np.sqrt(P_ran**2 + P_sys**2) 
    
        theta_pol = np.rad2deg(1/2* np.arctan2(u,q))
        theta_ran = np.rad2deg(1/2 * P_ran/P)
        theta_sys = np.rad2deg(1/2 * P_sys/P)
        theta_err = np.sqrt(theta_ran**2 + theta_sys**2)  

        if psANG == -999:
            pi = -999
            theta_r = -999
        elif psANG + 90 < 180:
            pi = psANG + 90
            theta_r = theta_pol - pi
            theta_r = theta_range(theta_r)
        else:
            pi = psANG - 90
            theta_r = theta_pol - pi
            theta_r = theta_range(theta_r)

        if P**2 - P_ran**2 < 0:
            Pcor = 0
            theta_ran = 51.96
            theta_sys = 51.96
            theta_err = 51.96
        else:
            Pcor = np.sqrt(P**2 - P_ran**2)
            theta_ran = np.rad2deg(1/2 * P_ran/Pcor)
            theta_sys = np.rad2deg(1/2 * P_sys/Pcor)
            theta_err = np.sqrt(theta_ran**2 + theta_sys**2) 

        if theta_r == -999:
            Pr = -999
        else:    
            Pr = Pcor * np.cos(2*np.deg2rad(theta_r))
            

        Pol_Log = pd.concat([Pol_Log,
                             pd.DataFrame({'filename':[file_name],
                                  'Object':[OBJECT],
                                  'Type':[Type],
                                  'JD':[JD_av],
                                  'Filter':[FILTER],
                                  'DATE':[Date_obs],
                                  'UT':[UT_obs],
                                  'alpha [deg]':[pA],
                                  'PsANG [deg]':[psANG],
                                  'Aper_radius [pix]':[aperture_radius],
                                  'Aper_scale':[ap],
                                  'FWHM_o':[FWHM_o],
                                  'FWHM_e':[FWHM_e],
                                  'q':[q],
                                  'u': [u],
                                  'eq':[eq],
                                  'eu':[eu],
                                  'ran_q':[q_ran],
                                  'ran_u':[u_ran],
                                  'sys_q':[q_sys],
                                  'sys_u':[u_sys],
                                  ##Raw-------------         
                                  'q_raw':[q_raw],
                                  'u_raw': [u_raw],
                                  'eq_raw':[np.sqrt(q_ran_raw**2+q_sys_raw**2)],
                                  'eu_raw':[np.sqrt(u_ran_raw**2+u_sys_raw**2)],
                                  'ran_q_raw':[q_ran_raw],
                                  'ran_u_raw':[u_ran_raw],
                                  'sys_q_raw':[q_sys_raw],
                                  'sys_u_raw':[u_sys_raw],         
                                           
                                  ##eff-------------         
                                  'q_eff':[q_eff],
                                  'u_eff': [u_eff],
                                  'eq_eff':[np.sqrt(q_ran_eff**2+q_sys_eff**2)],
                                  'eu_eff':[np.sqrt(u_ran_eff**2+u_sys_eff**2)],
                                  'ran_q_eff':[q_ran_eff],
                                  'ran_u_eff':[u_ran_eff],
                                  'sys_q_eff':[q_sys_eff],
                                  'sys_u_eff':[u_sys_eff],   
                                           
                                  ##inst-------------         
                                  'q_inst':[q_inst],
                                  'u_inst': [u_inst],
                                  'eq_inst':[np.sqrt(q_ran_inst**2+q_sys_inst**2)],
                                  'eu_inst':[np.sqrt(u_ran_inst**2+u_sys_inst**2)],
                                  'ran_q_inst':[q_ran_inst],
                                  'ran_u_inst':[u_ran_inst],
                                  'sys_q_inst':[q_sys_inst],
                                  'sys_u_inst':[u_sys_inst],
                                           
                                  ##off-------------         
                                  'q_off':[q_off],
                                  'u_off': [u_off],
                                  'eq_off':[np.sqrt(q_ran_off**2+q_sys_off**2)],
                                  'eu_off':[np.sqrt(u_ran_off**2+u_sys_off**2)],
                                  'ran_q_off':[q_ran_off],
                                  'ran_u_off':[u_ran_off],
                                  'sys_q_off':[q_sys_off],
                                  'sys_u_off':[u_sys_off],         
                                           
                                           
                                  'theta':[theta_pol],
                                  'theta_r':[theta_r],
                                  'eTheta':[theta_err],
                                           
                                  'P [%]':[P*100],
                                  'P_cor [%]':[Pcor*100],
                                  'eP [%]':[P_err*100],
                                  'Pr [%]':[Pr*100],
                                  'P_ran [%]':[P_ran*100],
                                  'P_sys [%]':[P_sys*100],
                                  'SNR':[snr],
                                  'AIRMASS':[AIRMASS],
                                  'delta':[delta],
                                  'r':[r], 'V':[V],
                                  'EXPTIME':[EXP],
                                  'flag':[flag_ls],
                                  'eff_cor':[eff_flag],
                                  'inst_cor':[inst_flag],
                                  'off_cor':[off_flag],
                                  'INSROT':[INSROT],
                                  'INROFF':[INROFF],
                                  'Tilt_ang':[tilted_ang]})])

    q_av, ranq_av = weight(Pol_Log['q'],Pol_Log['ran_q'])
    u_av, ranu_av = weight(Pol_Log['u'],Pol_Log['ran_u'])
    sysq_av = np.mean(Pol_Log['sys_q'])
    sysu_av = np.mean(Pol_Log['sys_u'])
    errq_av = (ranq_av**2 + sysq_av**2)**0.5
    erru_av = (ranu_av**2 + sysu_av**2)**0.5
    P = np.sqrt(q_av**2+u_av**2)
    ran_P = np.sqrt((q_av*ranq_av)**2 + (u_av*ranu_av)**2)/P
    sys_P = np.sqrt((q_av*sysq_av)**2 + (u_av*sysu_av)**2)/P
    eP = np.sqrt(ran_P**2 + sys_P**2)

    if P**2 - ran_P**2 < 0:
        Pcor = 0
        ran_PolAng = 51.96
        sys_PolAng = 51.96
        PolAng_error = 51.96
        # Naghizadeh-Khouei & Clarke 1993
    else:
        Pcor = np.sqrt(P**2 - ran_P**2)
        ran_PolAng = 1/2 * 180/3.14 * ran_P/Pcor
        sys_PolAng = 1/2 * 180/3.14 * sys_P/Pcor
        PolAng_error = np.sqrt(ran_PolAng**2 + sys_PolAng**2)  

    theta = 1/2*np.rad2deg(np.arctan2(u_av,q_av))
    theta = angle_set(theta)
    psang = np.mean(Pol_Log['PsANG [deg]'])

    if psang == -999:
        pi = -999
        theta_r = -999
    elif psang + 90 < 180:
        pi = psang + 90
        theta_r = theta - pi
        theta_r = theta_range(theta_r)
    else:
        pi = psang - 90
        theta_r = theta - pi
        
#         print('theta_pol',theta, 'pi',pi, 'theta_r',theta_r)
        theta_r = theta_range(theta_r)
#         print('theta_r',theta_r )

    if theta_r == -999:
        Pr = -999
    else:        
        Pr = Pcor*np.cos(2*np.deg2rad(theta_r))

    JD = np.mean(Pol_Log['JD'])
    alpha = np.mean(Pol_Log['alpha [deg]'])
    psang = np.mean(Pol_Log['PsANG [deg]'])
    snr_av = np.mean(Pol_Log['SNR'])
    delta_av = np.mean(Pol_Log['delta'])
    r_av = np.mean(Pol_Log['r'])
    tilted_ang_ = np.mean(Pol_Log['Tilt_ang'])
    UT_range = Pol_Log['UT'].values[0][:5] + '--'+Pol_Log['UT'].values[len(Pol_Log['UT'])-1][:5]
    AIRMASS_av = '{0:.1f}--{1:.1f}'.format(min(Pol_Log['AIRMASS'].values),max(Pol_Log['AIRMASS'].values))
    EXP_av = list(set(Pol_Log['EXPTIME'].values))
    if len(EXP_av)==1:
        EXP_av = EXP_av[0]

    file_name = 'Weighted_average'
    
    #Raw
    q_raw, ranq_raw = weight(Pol_Log['q_raw'],Pol_Log['ran_q_raw'])
    u_raw, ranu_raw = weight(Pol_Log['u_raw'],Pol_Log['ran_u_raw'])
    sysq_raw = np.mean(Pol_Log['sys_q_raw'])
    sysu_raw = np.mean(Pol_Log['sys_u_raw'])
    errq_raw = (ranq_raw**2 + sysq_raw**2)**0.5
    erru_raw = (ranu_raw**2 + sysu_raw**2)**0.5

    q_eff, ranq_eff = weight(Pol_Log['q_eff'],Pol_Log['ran_q_eff'])
    u_eff, ranu_eff = weight(Pol_Log['u_eff'],Pol_Log['ran_u_eff'])
    sysq_eff = np.mean(Pol_Log['sys_q_eff'])
    sysu_eff = np.mean(Pol_Log['sys_u_eff'])
    errq_eff = (ranq_eff**2 + sysq_eff**2)**0.5
    erru_eff = (ranu_eff**2 + sysu_eff**2)**0.5
    
    q_inst, ranq_inst = weight(Pol_Log['q_inst'],Pol_Log['ran_q_inst'])
    u_inst, ranu_inst = weight(Pol_Log['u_inst'],Pol_Log['ran_u_inst'])
    sysq_inst = np.mean(Pol_Log['sys_q_inst'])
    sysu_inst = np.mean(Pol_Log['sys_u_inst'])
    errq_inst = (ranq_inst**2 + sysq_inst**2)**0.5
    erru_inst = (ranu_inst**2 + sysu_inst**2)**0.5    
    

    
    Pol_Log = pd.concat([Pol_Log,
                         pd.DataFrame({'filename':[file_name],
                              'Object':[OBJECT],
                              'Type':[Type],
                              'JD':[JD],
                              'Filter':[FILTER],
                              'DATE':[Date_obs],
                              'UT':[UT_range],
                              'alpha [deg]':[alpha],
                              'PsANG [deg]':[psang],
                               ##Raw-------------         
                              'q_raw':[q_raw],
                              'u_raw': [u_raw],
                              'eq_raw':[errq_raw],
                              'eu_raw':[erru_raw],
                              'ran_q_raw':[ranq_raw],
                              'ran_u_raw':[ranu_raw],
                              'sys_q_raw':[sysq_raw],
                              'sys_u_raw':[sysu_raw],  
                                       
                               ##eff-------------                 
                              'q_eff':[q_eff],
                              'u_eff': [u_eff],
                              'eq_eff':[errq_eff],
                              'eu_eff':[erru_eff],
                              'ran_q_eff':[ranq_eff],
                              'ran_u_eff':[ranu_eff],
                              'sys_q_eff':[sysq_eff],
                              'sys_u_eff':[sysu_eff],                                        
                                       
                              ##Inst-------------         
                              'q_inst':[q_inst],
                              'u_inst': [u_inst],
                              'eq_inst':[errq_inst],
                              'eu_inst':[erru_inst],
                              'ran_q_inst':[ranq_inst],
                              'ran_u_inst':[ranu_inst],
                              'sys_q_inst':[sysq_inst],
                              'sys_u_inst':[sysu_inst],     

                                       
                              'q':[q_av],
                              'u':[u_av], 
                              'eq':[errq_av],
                              'eu':[erru_av],
                              'ran_q':[ranq_av],
                              'ran_u':[ranu_av],
                              'sys_q':[sysq_av],
                              'sys_u':[sysu_av],
                              'theta':[theta],
                              'theta_r':[theta_r],
                              'eTheta':[PolAng_error],
                              'P [%]':[P*100],
                              'P_cor [%]':[Pcor*100],
                              'eP [%]':[eP*100],
                              'Pr [%]':[Pr*100],
                              'P_ran [%]':[ran_P*100],
                              'P_sys [%]':[sys_P*100],
                              'SNR':[snr_av],
                              'AIRMASS':[AIRMASS_av],
                              'delta':[delta_av],
                              'r':[r_av],
                              'EXPTIME':[EXP_av],
                              'Aper_scale':[np.mean(df['Aper_scale'].values)],
                              'Tilt_ang':[tilted_ang_]})])



    Pol_name = ['filename','Object','Type','DATE','UT','JD', 'Filter','alpha [deg]',
                'P [%]','P_cor [%]', 'Pr [%]', 'eP [%]','theta_r','eTheta',
                'q', 'ran_q',  'sys_q','eq',
                'u','ran_u', 'sys_u', 'eu','theta',
                'AIRMASS', 'Aper_radius [pix]', 'Aper_scale', 'FWHM_e', 'FWHM_o', 
                'P_ran [%]', 'P_sys [%]','PsANG [deg]', 'SNR','delta','r','V','EXPTIME','flag','Type',
                'eff_cor','inst_cor','off_cor','INSROT','INROFF',
                'q_raw','u_raw','eq_raw','eu_raw','ran_q_raw','ran_u_raw','sys_q_raw','sys_u_raw',
                'q_eff','u_eff','eq_eff','eu_eff','ran_q_eff','ran_u_eff','sys_q_eff','sys_u_eff',
               'q_inst','u_inst','eq_inst','eu_inst','ran_q_inst','ran_u_inst','sys_q_inst','sys_u_inst',
               'q_off','u_off','eq_off','eu_off','ran_q_off','ran_u_off','sys_q_off','sys_u_off','Tilt_ang']
    Pol_Log = Pol_Log.reindex(columns=Pol_name)

    Pol_Log = Pol_Log.round({'alpha [deg]':2,  'P [%]':3,'P_cor [%]':3, 
                             'Pr [%]':3, 'eP [%]':3,'theta_r':3,'eTheta':2,'P [%]':3, 
                             'q':4, 'ran_q':4,  'sys_q':4,'eq':4,
                             'u':4, 'ran_u':4, 'sys_u':4, 'eu':4,
                             'theta':3,'AIRMASS':2, 'Aper_radius [pix]':1,
                             'FWHM_e':2, 'FWHM_o':2, 
                             'P_ran [%]':2, 'P_sys [%]':2,'PsANG [deg]':2, 'SNR':1,
                             'delta':3,'r':3, 'INSROT':2,'INROFF':2,'V':1,
                            'q_raw':4,'u_raw':4,'eq_raw':4,'eu_raw':4,
                             'ran_q_raw':4,'ran_u_raw':4,'sys_q_raw':4,'sys_u_raw':4,
                             'q_eff':4,'u_eff':4,'eq_eff':4,'eu_eff':4,
                             'ran_q_eff':4,'ran_u_eff':4,'sys_q_eff':6,'sys_u_eff':6,
                             'q_inst':4,'u_inst':4,'eq_inst':4,'eu_inst':4,'ran_q_inst':4,'ran_u_inst':4,'sys_q_inst':4,'sys_u_inst':4,
                             'q_off':4,'u_off':4,'eq_off':4,'eu_off':4,'ran_q_off':4,'ran_u_off':4,'sys_q_off':4,'sys_u_off':4,
                             'Tilt_ang':4})

    Date_obs = Date_obs.replace('-','_')
    if os.path.exists(os.path.join(subpath,'result'))==False:
        os.mkdir(os.path.join(subpath,'result'))    
    OBJECT = OBJECT.replace('/','_')
    if 'sky_' in phot:
        Pol_Log.to_csv(os.path.join(subpath,'result','sky_mPol_{0}.{3}.{1}.ap{2:.2f}.csv'.format(OBJECT,Date_obs,ap,FILTER)),
                      index=False)
        print(os.path.join(subpath,'result','sky_mPol_{0}.{3}.{1}.ap{2:.2f}.csv'.format(OBJECT,Date_obs,ap,FILTER)))
    else:
        Pol_Log.to_csv(os.path.join(subpath,'result','mPol_{0}.{3}.{1}.ap{2:.2f}.csv'.format(OBJECT,Date_obs,ap,FILTER)),
                      index=False)
        print(os.path.join(subpath,'result','mPol_{0}.{3}.{1}.ap{2:.2f}.csv'.format(OBJECT,Date_obs,ap,FILTER)))
    
def theta_range(theta):
    if theta < -20:
        theta = theta+180
        return theta_range(theta)
    elif theta > 180:
        theta = theta-180
        return theta_range(theta)
    else:
        return (theta)    
    
def weight(x,err):
    x = np.array(x)
    err = np.array(err)
    
    w = 1/err**2
    sumW = np.sum(w)
    weight = w/sumW
    
    xav = np.sum(weight*x)
    Err = 1/np.sqrt(sumW)
    
    return(xav,Err)

def circle(x,y,r):
    theta = np.linspace(0, 2*np.pi, 100)
    x1 = r*np.cos(theta)+y
    x2 = r*np.sin(theta)+x
    return(x2.tolist(),x1.tolist())

def angle_set(x,ang = 180):
    if -10 < x < 170:
        return x
    else:
        if x<45:
            x = x+ang
            return angle_set(x)
        if x>170:
            x = x-ang
            return angle_set(x) 

In [2]:
from astropy.io import ascii, fits
import numpy as np
import os
import glob
from astropy.time import Time
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
from astroquery.jplhorizons import HorizonsClass
from astroquery.jplhorizons import Horizons
from astropy.wcs import WCS

from astropy import units as u
import pandas as pd
import matplotlib as mpl
from MSI import MSI

import warnings
from astropy.io.fits.verify import VerifyWarning
warnings.simplefilter('ignore', category=VerifyWarning)
mpl.rc('figure', max_open_warning=0)


homepath = os.getenv('HOME')
 
subpath = os.path.join(homepath,'Dropbox/Research/test_pirka/95')
filelist =  glob.glob(os.path.join(subpath,'result/Photo_*.csv'))
filelist = sorted(filelist)
filelist

for phot in filelist:
    calib = './Calibration_info/MSI_pol_2023.csv'
    make_Pol_file(phot,calib,
                  eff=True,#Do efficiency calibration
                  inst=True,#Do instrument polarization calibration
                  off=True) #Do position angle offset calibration

/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23.ap1.40.csv
/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23.ap1.50.csv
/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23.ap1.60.csv
/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23.ap1.70.csv
/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23.ap1.80.csv
/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23.ap1.90.csv
/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23.ap2.00.csv
/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23.ap2.10.csv
/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23.ap2.20.csv
/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23.ap2.30.csv
/Users/judy/Dropbox/Research/test_pirka/95/result/mPol_95 Arethusa.Rc.2023_08_23

In [ ]:
file = os.path.join(homepath,'Dropbox/Research/test_pirka/95/result/Pol_95 Arethusa.Rc.2023_08_23.ap2.50.csv')
Pol_Log = pd.read_csv(file)
Log_ = Pol_Log[:-1]
plt.rcParams['font.size']=15
def weight(x,err):

    x = np.array(x)
    err = np.array(err)

    w = 1/err**2
    sumW = np.sum(w)
    weight = w/sumW

    xav = np.sum(weight*x)
    Err = 1/np.sqrt(sumW)

    return(xav,Err)
lim = 7*(abs(max(Log_['Pr [%]'].values) + max(Log_['eP [%]'].values))//5 + 1)
fig,ax = plt.subplots(1,1,figsize=(11.5,5.9))
for i in range(len(Log_)):

    color='k'
    flag = str(Log_['flag'].values[i])
    if 's' in flag:
        color='gray'
    if 'x' in flag:
        color='b'

    ax.errorbar(Log_['q'].values[i]*100, Log_['u'][i]*100,
                xerr = np.sqrt(Log_['sys_q'][i]**2 + Log_['ran_q'][i]**2)*100,
                yerr = np.sqrt(Log_['sys_u'][i]**2 + Log_['ran_u'][i]**2)*100,
               ls='',marker=r'${0}$'.format(i+1)
                ,ms=10,color=color,elinewidth=1)
lim = 4
ax.set_xlim(-lim,lim)
ax.set_ylim(-lim,lim)
ax.axvline(0,ls=':',color='k')
ax.axhline(0,ls=':',color='k')
q_av, ranq_av = weight(Log_['q'],Log_['ran_q'])
u_av, ranu_av = weight(Log_['u'],Log_['ran_u'])
sysq_av = np.mean(Log_['sys_q'])
sysu_av = np.mean(Log_['sys_u'])
errq_av = (ranq_av**2 + sysq_av**2)**0.5
erru_av = (ranu_av**2 + sysu_av**2)**0.5


P = np.sqrt(q_av**2+u_av**2)*100
ran_P = np.sqrt((q_av*ranq_av)**2 + (u_av*ranu_av)**2)/P*100
sys_P = np.sqrt((q_av*sysq_av)**2 + (u_av*sysu_av)**2)/P*100
eP = np.sqrt(ran_P**2 + sys_P**2)*100
P_cor = np.sqrt(P**2 - ran_P**2)
theta = 1/2*np.rad2deg(np.arctan2(u_av,q_av))
theta_r = theta - (np.mean(Log_['PsANG [deg]'])-90)
theta_ran = 1/2*180/3.14*ran_P/P_cor
theta_sys = 1/2*180/3.14*sys_P/P_cor
eTheta = np.sqrt(theta_ran**2 + theta_sys**2)
Pr = P_cor * np.cos(2*np.deg2rad(theta_r))
print(P * np.cos(2*np.deg2rad(theta_r))*100)
print('P = {0:.4f} ({1:.4f})'.format(P,eP))


sec_log = Pol_Log[Pol_Log['filename']=='Weighted_average']

q = q_av*100
u = u_av*100
eq = errq_av*100
eu = erru_av*100


q = sec_log['q'].values[0]*100
u = sec_log['u'].values[0]*100
eq = np.sqrt(sec_log['ran_q']**2 + sec_log['sys_q']**2).values[0]*100
eu = np.sqrt(sec_log['ran_u']**2 + sec_log['sys_u']**2).values[0]*100
P_cor = sec_log['P [%]'].values[0]
theta_r = sec_log['theta_r'].values[0]
eP = sec_log['eP [%]'].values[0]
eTheta = sec_log['eTheta'].values[0]
Pr = sec_log['Pr [%]'].values[0]
theta = sec_log['theta'].values[0]
alpha = sec_log['alpha [deg]'].values[0]

if abs(Pr)>100:
    Pr = np.nan
    theta_r = np.nan
if abs(alpha)>100: 
    alpha=np.nan
ax.errorbar(q, u, 
            xerr = eq,
            yerr = eu,
           ls='',marker='X',mfc='r',mec='r',ecolor='r',elinewidth=1,ms=13,
           label='{21}\
           \n\nDATE = {20}\
           \nUT = {19}\
            \nalpha = {13:.2f} deg\
           \n\nP = {8:.2f} ({9:.2f}) % \
           \ntheta = {10:.2f} ({11:.2f}) deg \
            \n\nPr = {0:.2f} ({1:.2f}) % \
           \ntheta_r = {2:.2f} ({3:.2f}) deg \
           \n\nq = {4:.2f} ({5:.2f}) \
           \nu = {6:.2f} ({7:.2f}) \
           \n\naper Dia = {12:.1f}*FWHM (~{14:.1f}pix)\
           \nSNR = {15:.1f}~{16:.1f} (for each images)\
           \n# of images = {17}\
           \nAirmass = {18}\
           \nFilter = {22}'.format(Pr,
                                        eP,
                                         theta_r,
                                         eTheta,
                                         q, eq,
                                         u, eu,
                                       P_cor, eP,
                                       theta,eTheta, 
                                      sec_log['Aper_scale'].values[0],alpha,2*np.mean(Log_['Aper_radius [pix]']),
                                             min(Log_['SNR']),max(Log_['SNR']),
                                   int(len(Log_)*4),
                                   sec_log['AIRMASS'].values[0],
                                   sec_log['UT'].values[0],
                                   sec_log['DATE'].values[0],
                                   sec_log['Object'].values[0],
                                  sec_log['Filter'].values[0]))
# print('P = {0:.4f} ({1:.4f})'.format(*weight(Log['u'],Log['eu']))
ax.set_xlabel('q')
ax.set_ylabel('u')
ax.set_title(sec_log['DATE'].values[0]+'/'+sec_log['Object'].values[0])
ax.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.tight_layout()


if os.path.exists(os.path.join(subpath,'result'))==False:
    os.mkdir(os.path.join(subpath,'result'))       
OBJECT_NAMGE= sec_log['Object'].values[0].replace('/','_')    
figname = os.path.join(subpath,'result','new_Fig_{0}.{1}.{3}.ap{2:.2f}.png'.format(OBJECT_NAMGE,
                                                              sec_log['DATE'].values[0],
                                                              sec_log['Aper_scale'].values[0],
                                                                              sec_log['Filter'].values[0]))

plt.savefig(figname)

figname = os.path.join(subpath,'result','m_Fig_{0}.{1}.{3}.ap{2:.2f}.pdf'.format(OBJECT_NAMGE,
                                                              sec_log['DATE'].values[0],
                                                              sec_log['Aper_scale'].values[0],
                                                                              sec_log['Filter'].values[0]))

plt.savefig(figname)
plt.show()

    
print(r'''
---------------
{0} 
---------------
{1} {2}
Pr = {3:.2f} ({4:.2f})%
theta_r = {5:.2f} ({6:.2f})deg
SNR = {7}~{8} (for each images)
# of images = {9}
---------------

'''.format(OBJECT_NAMGE,sec_log['DATE'].values[0],sec_log['Filter'].values[0],
          Pr,eP, theta_r,eTheta,
           min(Log_['SNR']),max(Log_['SNR']),int(len(Log_)*4)))
